In [1]:
CODE_REVIEWER = """
# IDENTITY

You are an AI named Codia. You have extensive knowledge and skill in programming languages, especially Python. You are aware of the best practices used in programming, have an extensive extensive experience in lagorithms, data structures and overall computer science.

You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model.

# GOALS

Your task is to evaluate and provide feedback for a conversation between a human user and an AI Assistant that is based on the latest large language model architecture.
Focus of your evaluation is code in the replies generated by the AI Assistant only. The conversation environment is a Jupyter notebook, thus things that are run in other cells, are available in the next cells.

# RULES

Attributes to consider:
- Code Correctness
- Code Efficiency
- Best Practices
- Code Readability
- Code style Consistency
- Code purpose and usefulness for user request satisfaction

**1. Identification of Code for Review**
- Target for analysis: Code generated by the LLM Assistant in a reply to the User within a Jupyter notebook exchange.
- Exclude analysis of human user input for focused improvement on LLM-generated content.
- Exclude LLM Assistant text content that is not related to the code, only review code snippets and code cells. Text is for context and reasoning/explanation only, you can assess meaning of the text in relation to the code.
- Exclude concerns about code explanation in the text parts if they are not comments inside the code, as it will be covered by other reviewers.

**2. Evaluation Criteria Definitions**
- Correctness: The code must be devoid of bugs and errors.
- Efficiency: The code must be optimized for maximum performance.
- Best Practices: The code must adhere to established programming conventions, techniques, and guidelines.
- Readability: The code must be easily comprehensible, with suitable naming conventions and comments where complexity demands.
- Consistency: The code must be consistent with the Assistant's programming identity and the context of the user interaction.
- Completeness of the conversation as a whole: was user request satisfied or does conversation still needs more interactions(very bad)?

**3. Review Guidelines**
- Avoid general praise observations: Be specific and objective in your feedback.
- Avoid nitpicky/subjective criticism: Focus on substantial issues that affect the code quality.

# OUTPUT_FORMAT

Your output must be a JSON object with the following fields:

# critical_severity_issues - list of issues that decrease usefulness of the Assistant code replies for the human user in major ways.
# medium_severity_issues - list of issues that have a strong influence on the conversation flow and usefulness.
# could_have_been_better_issues - list of issues that are suggestions that have almost no influence on the score
Each issue should point out a specific problem and how to fix it in exec summary fashion. Start each issue with the cell position it was found in and a short piece of text to find specific place of the issue.
Each issue consists of an `issue_dict` - should contain the following fields: `cell_position`, `what` is the issue, `why` it's an issue, `where` in the cell, how to `fix` it.

# scoring_explanation - using rubrics, concisly point out the logic behind scoring this concersation.

# score - a number between 1 and 5 that ranges from the quality of the code, where 1 is the worst and 5 is the best, based on the criteria outlined in the grading rubric.

### 5 - Excellent
- Well Formatted
- Correct
- Optimal
- Highly readable
- Useful
- conversation must be complete ending in user request full satisfaction

### 4 - Good
- Correct but can be slightly optimized in terms of approach / speed / readability

### 3 - Acceptable
- The code is correct but can be significantly improved.
- The code is not readable.

### 2 - Needs Improvement
- The code is incorrect / out of scope / has syntax errors.
- Looks like it’s copied from ChatGPT - robotic, no personality, inhuman.

### 1 - Poor
- Incomplete or missing Code, but is required or implied by context of the interaction to make it useful aka did not satisfy user's request and desire

```json
{
    "critical_severity_issues": list[issue_dict],
    "medium_severity_issues": list[issue_dict],
    "could_have_been_better_issues": list[issue_dict],
    "scoring_explanation": str,
    "score": number
}
```

# REFOCUS:
- You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
- If the code was **unnecessary** aka user request FULLY satisfied without it, it can be absent and thus must receive null.
- If code from assistant is necessary by the conversation flow to satisfy user's request but it is not there - score it as 1, do not mark as 5.
"""

In [2]:
import utils

from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from src.llm_reviewer.notebook_reviewer import IssueLevel
from src.llm_reviewer.llm_api import global_usage_manager
global_usage_manager.reset_usage()

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:

def make_review_request(nb):
    llm_client = LLMAPIFactory().get()
    cells_with_numbers = [{'cell_position': i + 1, **c,} for i, c in enumerate(nb['messages'])]
    r = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': CODE_REVIEWER}, {'role': 'user', 'content': f'# CONVERSATION_START\n\n\n{cells_with_numbers}\n\n\n# CONVERSATION_END'}],
        'gpt-4-1106-preview',
        temperature=0.0,
        max_tokens=4000,
        response_format={"type": "json_object"},
        retries=3,
        seed=42
    )
    return r

In [4]:
from IPython.display import display, Markdown

def format_review_sections(data):
    sections = ["critical_severity_issues", "medium_severity_issues", "scoring_explanation", "score"]
    markdown_output = ""
    for section in sections:
        if section in data:
            if isinstance(data[section], list):
                markdown_output += f"# {section.replace('_', ' ').title()}\n"
                for issue in data[section]:
                    issue_details = f"**Cell {issue['cell_position']}**: {issue['what']}\n"
                    issue_details += f"- **Why**: {issue['why']}\n"
                    issue_details += f"- **Where**: {issue['where']}\n"
                    issue_details += f"- **Fix**: {issue['fix']}\n\n"
                    markdown_output += issue_details
            else:
                if section == "score":
                    markdown_output += f"# Score: {data[section]}\n\n"
                else:
                    markdown_output += f"# {section.replace('_', ' ').title()}\n{data[section]}\n\n"
        else:
            raise Exception(f'{section=} not found')
    return markdown_output

def print_as_markdown(data):
    markdown_output = format_review_sections(data)
    display(Markdown(markdown_output))


In [ ]:
r = make_review_request(notebook)
print_as_markdown(r)

In [5]:
def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    timestamp = pd.to_datetime(timestamp).date()

In [7]:
import pandas as pd

In [6]:
pd.to_datetime("12/20/2023 21:01:11").date()

datetime.date(2023, 12, 20)

In [6]:
from pytz import timezone

# Assuming the string is already in GMT+2
timestamp = pd.to_datetime("12/20/2023 21:01:11").tz_localize(timezone('Etc/GMT-2'))
timestamp

NameError: name 'pd' is not defined

In [7]:
from src.sheets_utils import download_sheet_as_df
from utils import PROJECT_ROOT
service_account_path = PROJECT_ROOT + "creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
reviews_sheets_df = download_sheet_as_df(service_account_path, tracking_sheet_id, 'Reviews')
from utils import get_delivered_df
delivered_df = get_delivered_df([1,2,3,4,5])

In [8]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timezone
import pandas as pd
from src.gdrive_api.auth import build_service
from datetime import timedelta

utc_timezone = timezone.utc
gmt_plus_2h_timezone = timezone(timedelta(hours=2))


drive_service = build_service(service_account_path)
def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    
    timestamp = pd.to_datetime(timestamp).tz_localize(gmt_plus_2h_timezone)

    # Initialize the latest revision before the timestamp as None
    latest_revision_before_timestamp = None

    for revision in revisions['revisions']:
        # Convert the modifiedTime of the revision to a datetime object
        modified_time = pd.to_datetime(revision['modifiedTime'])
        # If the modifiedTime is before the timestamp
        if modified_time <= timestamp:
            # If this is the first revision or this revision is later than the latest found so far
            if latest_revision_before_timestamp is None or modified_time > pd.to_datetime(latest_revision_before_timestamp['modifiedTime']):
                # Update the latest revision before the timestamp
                latest_revision_before_timestamp = revision

    return latest_revision_before_timestamp



def get_file_id_from_task_link(task_link):
    try:
        return task_link.split("/")[-1].split('#')[0]
    except Exception as e:
        print('ERROR' + '='*60)
        print(task_link)
        return None

## Add a new column to the DataFrame for the file IDs
#selected_rows_df['file_id'] = selected_rows_df['Task Link [Google Colab]'].apply(get_file_id_from_task_link)

# Apply the function to each row in the DataFrame
#selected_rows_df['revision'] = selected_rows_df.apply(lambda row: get_revision_before_timestamp(row['file_id'], row['Timestamp']) if row['file_id'] is not None else None, axis=1)

In [9]:
def append_revision_id_to_task_link(task_link, revision):
    if revision is not None:
        revision_id = revision.get('id')
        return f"{task_link}#revisionId={revision_id}"
    return task_link

In [11]:


selected_rows_df['task_link_rev'] = selected_rows_df.apply(lambda row: append_revision_id_to_task_link(row['Task Link [Google Colab]'], row['revision']), axis=1)


,Timestamp,Email Address,Task Link [Google Colab],Code Quality,Language Quality,Feedback for the Trainer,Author Email,file_id,revision,task_link_rev
1,12/19/2023 12:01:41,youssef.h@turing.com,https://colab.research.google.com/drive/1ZSjvo...,4,4,"The header of the code cell should be ""# Assis...",None,1ZSjvocSfnuFQY-8gGSggGUEtfnLfS_B4,{'id': '0BxmKQQ91GHhcalFrU1l4ZHppYlVtUnQ2VFptN...,https://colab.research.google.com/drive/1ZSjvo...
2,12/19/2023 12:09:35,youssef.h@turing.com,https://colab.research.google.com/drive/1nv28F...,5,3,"++The code is correct, concise and well struct...",None,1nv28F9bJOHxEXJUaLQv3V9z8Fj8SHzmV,{'id': '0BxmKQQ91GHhcdlFMRFF6YkcxNDhzZzlOUDhsa...,https://colab.research.google.com/drive/1nv28F...
3,12/19/2023 12:31:33,youssef.h@turing.com,https://colab.research.google.com/drive/1nBjh1...,4,5,++ Code is clean and well structured\n++Langua...,None,1nBjh1lLJuZX9kIIdVaHQPrHnWF4tx7Sg,{'id': '0BxmKQQ91GHhcN1JSYXVuOTl2U0M1T1N0ZWgrc...,https://colab.research.google.com/drive/1nBjh1...
58,12/23/2023 14:38:04,youssef.h@turing.com,https://colab.research.google.com/drive/17eg2v...,4,5,This looks good.\n\nThe code could be improved...,caram.v@turing.com,17eg2v1KiqRXymrzVdky6sni_jesoQzAu,{'id': '0BxmKQQ91GHhcc1VkZXVuNGRrcStDbVpzMC81V...,https://colab.research.google.com/drive/17eg2v...
59,12/23/2023 14:40:57,youssef.h@turing.com,https://colab.research.google.com/drive/1XOkKN...,5,5,Very Good!,caram.v@turing.com,1XOkKNQnu93JN5bIFAMe6W-8NdfF5Qo0-,{'id': '0BxmKQQ91GHhcTGxHZll5aERGdmJhbkxhQThGV...,https://colab.research.google.com/drive/1XOkKN...
60,12/23/2023 14:47:27,youssef.h@turing.com,https://colab.research.google.com/drive/1VmX6t...,5,3,--The first assistant markdown has multiple gr...,rohit.kumar@turing.com,1VmX6t6BuosB-HooXaYOgMgW0_uuzZ42z,{'id': '0BxmKQQ91GHhcZUJmZXJCcnhUT0UyQzhnaDBta...,https://colab.research.google.com/drive/1VmX6t...
61,12/23/2023 14:56:30,youssef.h@turing.com,https://colab.research.google.com/drive/1CQSKk...,3,3,--Conversation does not meet the minimum lengt...,rohit.kumar@turing.com,1CQSKk-8ceLdmyBUKHT16aM2unS4U5Tgu,{'id': '0BxmKQQ91GHhcQVpIZGl3Zzh4V1BCenU2dkhtW...,https://colab.research.google.com/drive/1CQSKk...
8,12/20/2023 19:38:33,chirag.rade-c@turing.com,https://colab.research.google.com/drive/11CEko...,5,4,++ Code is clean and well structured \n++ Test...,hriday.thakkar@turing.com,11CEkoZ0S3dbU1-2Cp5b1YTZf0HkI8ljV,{'id': '0BxmKQQ91GHhcMGhnQkFyZVB3MnhiOWxvM1Ixd...,https://colab.research.google.com/drive/11CEko...
9,12/20/2023 19:55:29,chirag.rade-c@turing.com,https://colab.research.google.com/drive/1dsE7l...,3,4,-- Incorrect use of f-string in the last print...,ali.nauman@turing.com,1dsE7ljA8jt4lvYHdc60Kdtme2Y829m41,{'id': '0BxmKQQ91GHhcekVjUFF3UTlrOEtEVkVNc1VCU...,https://colab.research.google.com/drive/1dsE7l...
10,12/20/2023 19:57:38,chirag.rade-c@turing.com,https://colab.research.google.com/drive/1wEgUF...,5,4,++ code is clean and well structured\n++ comme...,riya.dhar@turing.com,1wEgUF08nHpAhTSvWaDR8uPnrDzw1PnEm,{'id': '0BxmKQQ91GHhccWYyYzFnRzhpRGtWY3B0VXBmW...,https://colab.research.google.com/drive/1wEgUF...


In [10]:

import utils
from importlib import reload
import jsonl_dump
reload(jsonl_dump)
from jsonl_dump import download_and_parse_notebook

In [16]:

parsed_notebooks = []
for i, (index, row) in enumerate(selected_rows_df.iterrows()):
    file_id = row['file_id']
    revision_id = row['revision'].get('id') if row['revision'] is not None else None
    print(f"Downloading and parsing notebook {i+1}/{len(selected_rows_df)} (File ID: {file_id}, Revision ID: {revision_id})")
    parsed_notebook = download_and_parse_notebook(service_account_path, file_id, revision_id)
    parsed_notebooks.append(parsed_notebook)


Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...


In [18]:
parsed_notebooks[0]['messages']

[{'role': 'User',
  'content': "What's the Python code for a segment tree to handle range queries and updates?",
  'type': 'markdown'},
 {'role': 'Assistant',
  'content': 'Here is a `SegmentTree` class with methods to build the tree, update a value, and query a range sum.',
  'type': 'markdown'},
 {'role': 'Assistant',
  'content': '# LLM\n\nclass SegmentTree:\n    def __init__(self, nums):\n        self.n = len(nums)\n        self.tree = [0] * (4 * self.n)\n        self.build_tree(nums, 0, 0, self.n - 1)\n\n    def build_tree(self, nums, tree_index, lo, hi):\n        if lo == hi:\n            self.tree[tree_index] = nums[lo]\n            return\n        mid = (lo + hi) // 2\n        left_child = 2 * tree_index + 1\n        right_child = 2 * tree_index + 2\n        self.build_tree(nums, left_child, lo, mid)\n        self.build_tree(nums, right_child, mid + 1, hi)\n        self.tree[tree_index] = self.tree[left_child] + self.tree[right_child]\n\n    def update(self, index, val):\n     

In [21]:
for i, notebook in enumerate(parsed_notebooks):
    print(f"Reviewing notebook {i+1}/{len(parsed_notebooks)}")
    # Perform the review for each notebook's messages
    review = make_review_request(notebook)
    # Store the review in the notebook dictionary
    notebook['raw_review'] = review
    # Format the review sections into a string and store it
    formatted_review = format_review_sections(review)
    notebook['formatted_review'] = formatted_review


Reviewing notebook 1/27
Reviewing notebook 2/27
Reviewing notebook 3/27
Reviewing notebook 4/27
Reviewing notebook 5/27
Reviewing notebook 6/27
Reviewing notebook 7/27
Reviewing notebook 8/27
Reviewing notebook 9/27
Reviewing notebook 10/27
Reviewing notebook 11/27
Reviewing notebook 12/27
Reviewing notebook 13/27
Reviewing notebook 14/27
Reviewing notebook 15/27
Reviewing notebook 16/27
Reviewing notebook 17/27
Reviewing notebook 18/27
Reviewing notebook 19/27
Reviewing notebook 20/27
Reviewing notebook 21/27
Reviewing notebook 22/27
Reviewing notebook 23/27
Reviewing notebook 24/27
Reviewing notebook 25/27
Reviewing notebook 26/27
Reviewing notebook 27/27


In [10]:
from src.llm_reviewer.llm_api import global_usage_manager
global_usage_manager.print_costs()

Total cost for all models: $0.000


In [26]:
print(parsed_notebooks[0]['formatted_review'])

# Critical Severity Issues
# Medium Severity Issues
**Cell 3**: The `SegmentTree` class lacks a method to handle lazy propagation for range updates.
- **Why**: Without lazy propagation, the update method is inefficient for range updates as it updates each node individually, leading to a time complexity of O(n) for each update instead of O(log n).
- **Where**: In the `SegmentTree` class definition.
- **Fix**: Implement a method for lazy propagation to handle range updates efficiently.

# Scoring Explanation
The code provided in cell 3 is correct and follows good practices in terms of readability and style consistency. However, it lacks an efficient way to handle range updates, which is a significant part of segment tree functionality. The absence of lazy propagation means that the update method is not optimized for performance when dealing with range updates. This issue is of medium severity as it affects the efficiency of the code but does not render it incorrect. The code is well-form

In [27]:
parsed_notebooks[0]['colab_link']

'https://colab.research.google.com/drive/1ZSjvocSfnuFQY-8gGSggGUEtfnLfS_B4?revisionId=0BxmKQQ91GHhcalFrU1l4ZHppYlVtUnQ2VFptNTAzVUpGNnhNPQ'

In [54]:
print(notebooks_df.iloc[2]['full_history_formatted_review'])

# Critical Severity Issues
**Cell 3**: The `add_edge` function is incorrectly updating the adjacency list.
- **Why**: The function creates a new `neighbours_list` every time it is called, which does not maintain the existing edges for a node.
- **Where**: In the `add_edge` function.
- **Fix**: Instead of creating a new list, directly append the new neighbor to the adjacency list for the node if it exists, or create a new entry if it does not.

# Medium Severity Issues
**Cell 3**: The `add_node` and `add_edge` functions do not handle the case where a node is added with an edge simultaneously.
- **Why**: If a user tries to add an edge with a new node that is not in `node_list`, the edge will not be added, and the user is not informed why.
- **Where**: In the `add_node` and `add_edge` functions.
- **Fix**: Modify the `add_edge` function to add nodes to `node_list` if they are not already present before creating an edge.

**Cell 3**: The `graph` function is not optimally named and its purp

In [52]:
import pandas as pd

notebooks_df = pd.DataFrame(parsed_notebooks)
notebooks_df['full_history_code_score'] = notebooks_df['raw_review'].apply(lambda x: x.get('score') if x is not None else None)
notebooks_df.rename(columns={'formatted_review': 'full_history_formatted_review'}, inplace=True)
notebooks_df = notebooks_df.merge(selected_rows_df[['file_id', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']], left_on='id', right_on='file_id', how='left')

notebooks_df = notebooks_df[['colab_link', 'full_history_formatted_review', 'full_history_code_score', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']]

notebooks_df.rename(columns={'score': 'selected_rows_score', 'Code Quality':'Human Code Quality', 'Language Quality':'Human Language Quality', 'Feedback for the Trainer': 'Human Reviewer Feedback'}, inplace=True)
notebooks_df['colab_link'] = notebooks_df['colab_link'].str.replace('?', '#')
from utils import DATA_DIR
notebooks_df.to_csv(DATA_DIR + 'full_hsitory_df_code_only.csv', index=False)
notebooks_df

,colab_link,full_history_formatted_review,full_history_code_score,Human Code Quality,Human Language Quality,Human Reviewer Feedback
0,https://colab.research.google.com/drive/1ZSjvo...,# Critical Severity Issues\n# Medium Severity ...,3,4,4,"The header of the code cell should be ""# Assis..."
1,https://colab.research.google.com/drive/1nv28F...,# Critical Severity Issues\n# Medium Severity ...,3,5,3,"++The code is correct, concise and well struct..."
2,https://colab.research.google.com/drive/1nBjh1...,# Critical Severity Issues\n**Cell 3**: The `a...,2,4,5,++ Code is clean and well structured\n++Langua...
3,https://colab.research.google.com/drive/17eg2v...,# Critical Severity Issues\n# Medium Severity ...,4,4,5,This looks good.\n\nThe code could be improved...
4,https://colab.research.google.com/drive/1XOkKN...,# Critical Severity Issues\n# Medium Severity ...,4,5,5,Very Good!
5,https://colab.research.google.com/drive/1VmX6t...,# Critical Severity Issues\n# Medium Severity ...,4,5,3,--The first assistant markdown has multiple gr...
6,https://colab.research.google.com/drive/1CQSKk...,# Critical Severity Issues\n# Medium Severity ...,4,3,3,--Conversation does not meet the minimum lengt...
7,https://colab.research.google.com/drive/11CEko...,# Critical Severity Issues\n# Medium Severity ...,4,5,4,++ Code is clean and well structured \n++ Test...
8,https://colab.research.google.com/drive/1dsE7l...,# Critical Severity Issues\n# Medium Severity ...,5,3,4,-- Incorrect use of f-string in the last print...
9,https://colab.research.google.com/drive/1wEgUF...,# Critical Severity Issues\n# Medium Severity ...,4,5,4,++ code is clean and well structured\n++ comme...


In [46]:
notebooks_df

,colab_link,full_history_formatted_review,full_history_code_score,Human Code Quality,Human Language Quality
0,https://colab.research.google.com/drive/1ZSjvo...,# Critical Severity Issues\n# Medium Severity ...,3,4,4
1,https://colab.research.google.com/drive/1nv28F...,# Critical Severity Issues\n# Medium Severity ...,3,5,3
2,https://colab.research.google.com/drive/1nBjh1...,# Critical Severity Issues\n**Cell 3**: The `a...,2,4,5
3,https://colab.research.google.com/drive/17eg2v...,# Critical Severity Issues\n# Medium Severity ...,4,4,5
4,https://colab.research.google.com/drive/1XOkKN...,# Critical Severity Issues\n# Medium Severity ...,4,5,5
5,https://colab.research.google.com/drive/1VmX6t...,# Critical Severity Issues\n# Medium Severity ...,4,5,3
6,https://colab.research.google.com/drive/1CQSKk...,# Critical Severity Issues\n# Medium Severity ...,4,3,3
7,https://colab.research.google.com/drive/11CEko...,# Critical Severity Issues\n# Medium Severity ...,4,5,4
8,https://colab.research.google.com/drive/1dsE7l...,# Critical Severity Issues\n# Medium Severity ...,5,3,4
9,https://colab.research.google.com/drive/1wEgUF...,# Critical Severity Issues\n# Medium Severity ...,4,5,4


In [11]:
notebook_urls = """
https://colab.research.google.com/drive/1_lUJ5N3a8gX4pz2AEZ1LHhlLxKc-RsRG
https://colab.research.google.com/drive/1qJPPqc7d1M4ljB2fWss3XniuCXL_-J6w
https://colab.research.google.com/drive/1UeXB6ceNocGOSwSsOxH3ZQ2ZPFCBsQQn
https://colab.research.google.com/drive/1cP6qzbSAWYKL8QgTNPa9S85qApJejzde
https://colab.research.google.com/drive/1FIo_t-nNB41OVE4BqsfHC8EeMKcB3hVd
https://colab.research.google.com/drive/1JUY9363HPsn5ChuweH_i31EzSsJPzSwX
https://colab.research.google.com/drive/1kL_7R8kTnbT0nDuuVweaHMszel7ITPhO
https://colab.research.google.com/drive/1yfP7u0DJjcZvthfDq-_GQcc343eTRMWM
https://colab.research.google.com/drive/182EqzbvKd4W7-eEuAoRInPtWzMj9oVPl
https://colab.research.google.com/drive/1CPEz-cjdkWop8Ju5Mn0Gb75sEr24UUUl
https://colab.research.google.com/drive/189Fnel9xYNfdHXLDhwp83We0qiHvTcG-
https://colab.research.google.com/drive/1KtSOMVTE5kj8VuId-zm-y_TZVVMQwYwZ
https://colab.research.google.com/drive/1Unbsf2xICxBcqMFzYBbyS9NW4kVtr7Os
""".strip().split()
len(notebook_urls)

13

In [10]:
df_rev

NameError: name 'reviews_df' is not defined

In [12]:
reviews_sheets_df

,Timestamp,Email Address,Task Link [Google Colab],Code Quality,Language Quality,Feedback for the Trainer,Author Email
0,None,None,None,None,None,None,None
1,12/19/2023 12:01:41,youssef.h@turing.com,https://colab.research.google.com/drive/1ZSjvo...,4,4,"The header of the code cell should be ""# Assis...",None
2,12/19/2023 12:09:35,youssef.h@turing.com,https://colab.research.google.com/drive/1nv28F...,5,3,"++The code is correct, concise and well struct...",None
3,12/19/2023 12:31:33,youssef.h@turing.com,https://colab.research.google.com/drive/1nBjh1...,4,5,++ Code is clean and well structured\n++Langua...,None
4,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...
514,1/16/2024 15:04:17,freitas.g@turing.com,https://colab.research.google.com/drive/1_TyU5...,5,5,++ very good,khalid.s@turing.com
515,1/16/2024 15:10:21,freitas.g@turing.com,https://colab.research.google.com/drive/1oSndl...,5,5,++ very well done\n-- feel like the questions ...,zain.v@turing.com
516,1/16/2024 15:14:43,freitas.g@turing.com,https://colab.research.google.com/drive/12soeb...,5,4,\n++ very well done\n-- feel like it's a bit t...,caram.v@turing.com
517,1/16/2024 16:13:00,freitas.g@turing.com,https://colab.research.google.com/drive/1FKh_F...,5,5,++ very well done,elsadek.a@turing.com


In [13]:
selected_rows_df = delivered_df[delivered_df['task_link'].isin(notebook_urls)]
selected_rows_df = selected_rows_df.merge(reviews_sheets_df, left_on='task_link', right_on='Task Link [Google Colab]', how='left')

selected_rows_df

,task_link,jsonl_link,metadata__topic,number_of_turns,duration_mins,batch_id,Timestamp,Email Address,Task Link [Google Colab],Code Quality,Language Quality,Feedback for the Trainer,Author Email
0,https://colab.research.google.com/drive/1UeXB6...,https://drive.google.com/file/d/1GpwZJpZdS-QdH...,database > transaction_management,2,40,4,1/5/2024 18:31:37,zain.v@turing.com,https://colab.research.google.com/drive/1UeXB6...,3,3,"- missing code example in first turn, a sample...",kumbar.r@turing.com
1,https://colab.research.google.com/drive/1qJPPq...,https://drive.google.com/file/d/17faYUO_xOnLan...,NaN,1,NaN,5,1/4/2024 19:52:29,zain.v@turing.com,https://colab.research.google.com/drive/1qJPPq...,1,1,--It's empty.,icaro.c@turing.com
2,https://colab.research.google.com/drive/1cP6qz...,https://drive.google.com/file/d/12lm8x2QikaLaZ...,NaN,1,NaN,5,1/7/2024 21:16:11,ruturaj.m@turing.com,https://colab.research.google.com/drive/1cP6qz...,2,1,-- No Explanation\n-- No Headers\n-- No Comments,patelia.a@turing.com
3,https://colab.research.google.com/drive/1CPEz-...,https://drive.google.com/file/d/15d7J16byuhmY5...,NaN,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://colab.research.google.com/drive/189Fne...,https://drive.google.com/file/d/1RhtPfMjxsa_n2...,NaN,8,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://colab.research.google.com/drive/1Unbsf...,https://drive.google.com/file/d/1-XsTKRyBDpGa_...,NaN,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://colab.research.google.com/drive/1KtSOM...,https://drive.google.com/file/d/1S7BfTUH4dfej9...,NaN,4,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://colab.research.google.com/drive/1_lUJ5...,https://drive.google.com/file/d/1DbVKF30oMVp7V...,NaN,1,NaN,5,1/4/2024 16:21:26,freitas.g@turing.com,https://colab.research.google.com/drive/1_lUJ5...,1,1,-- assistant header is wrong it should be **As...,gautam.g@turing.com
8,https://colab.research.google.com/drive/1JUY93...,https://drive.google.com/file/d/1gZERrrCwAFqEF...,NaN,1,NaN,5,1/8/2024 18:18:54,freitas.g@turing.com,https://colab.research.google.com/drive/1JUY93...,2,2,"-- really low effort, I understand that someti...",pawan.s@turing.com
9,https://colab.research.google.com/drive/1kL_7R...,https://drive.google.com/file/d/11t58DSUzw2nXr...,NaN,2,NaN,5,1/8/2024 20:29:59,freitas.g@turing.com,https://colab.research.google.com/drive/1kL_7R...,1,1,-- The user question is clearly a copy and pas...,pawan.s@turing.com


In [14]:

# Add a new column to the DataFrame for the file IDs
selected_rows_df['file_id'] = selected_rows_df['task_link'].apply(get_file_id_from_task_link)

# Apply the function to each row in the DataFrame
selected_rows_df['revision'] = selected_rows_df.apply(lambda row: get_revision_before_timestamp(row['file_id'], row['Timestamp']) if row['file_id'] is not None else None, axis=1)
selected_rows_df['task_link_rev'] = selected_rows_df.apply(lambda row: append_revision_id_to_task_link(row['task_link'], row['revision']), axis=1)

parsed_notebooks = []
for i, (index, row) in enumerate(selected_rows_df.iterrows()):
    file_id = row['file_id']
    revision_id = row['revision'].get('id') if row['revision'] is not None else None
    print(f"Downloading and parsing notebook {i+1}/{len(selected_rows_df)} (File ID: {file_id}, Revision ID: {revision_id})")
    parsed_notebook = download_and_parse_notebook(service_account_path, file_id, revision_id)
    parsed_notebooks.append(parsed_notebook)


Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


In [16]:
import os
import pickle
from utils import DATA_DIR

# Create directory if it does not exist
selected_notebooks_dir = os.path.join(DATA_DIR, 'selected_for_eval_notebooks')
os.makedirs(selected_notebooks_dir, exist_ok=True)

# Pickle dump the parsed_notebooks
with open(os.path.join(selected_notebooks_dir, 'parsed_notebooks.pkl'), 'wb') as f:
    pickle.dump(parsed_notebooks, f)



In [ ]:
import os
import pickle
from utils import DATA_DIR

# Pickle load the parsed_notebooks
with open(os.path.join(selected_notebooks_dir, 'parsed_notebooks.pkl'), 'rb') as f:
    parsed_notebooks = pickle.load(f)


In [17]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

def review_notebook(notebook):
    review = make_review_request(notebook)
    formatted_review = format_review_sections(review)
    return review, formatted_review

def review_all_notebooks(parsed_notebooks):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(review_notebook, notebook): i for i, notebook in enumerate(parsed_notebooks)}
        progress_bar = tqdm(as_completed(futures), total=len(parsed_notebooks), desc="Reviewing Notebooks", leave=False)
        for future in progress_bar:
            review, formatted_review = future.result()
            notebook_index = futures[future]
            parsed_notebooks[notebook_index]['raw_review'] = review
            parsed_notebooks[notebook_index]['formatted_review'] = formatted_review
        progress_bar.close()
    return parsed_notebooks


In [18]:
for p in parsed_notebooks:
    if '1JUY9' in p['colab_link']:
        print(p)

pp = review_all_notebooks([p for p in parsed_notebooks if '1JUY9' in p['colab_link']])

{'id': '1JUY9363HPsn5ChuweH_i31EzSsJPzSwX', 'colab_link': 'https://colab.research.google.com/drive/1JUY9363HPsn5ChuweH_i31EzSsJPzSwX', 'revision_id': '0BxmKQQ91GHhcN3ViTitrMVh2Vi82V3l6a2tpaGFWWnBLYlhZPQ', 'metadata': {'topic': 'debugging_and_tracing', 'type': 'query'}, 'messages': [{'role': 'User', 'content': 'Hi, I am getting error running the code: The generated image exceeds the limit. How to solve it?\n```python\nPaddleHub2.4.0, PaddlePaddle2.5.2\nSystem environment: Windows\npython version: 3.9.18\nError report: In the prediction code example, an error is reported after running images = module.generate_image(text_prompts=text_prompts, style=\'oil painting\', output_dir=\'./ernie_vilg_out/\')\nTraceback (most recent call last):\nFile "", line 1, in\nFile "C:\\Users\\k\\anaconda3\\envs\\wm\\lib\\site-packages\\paddlehub\\compat\\paddle_utils.py", line 221, in runner\nreturn func(*args, **kwargs)\nFile "C:\\Users\\k.paddlehub\\modules\\ernie_vilg\\module.py", line 121, in generate_im

Reviewing Notebooks:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
pp

[{'id': '1JUY9363HPsn5ChuweH_i31EzSsJPzSwX',
  'colab_link': 'https://colab.research.google.com/drive/1JUY9363HPsn5ChuweH_i31EzSsJPzSwX',
  'revision_id': '0BxmKQQ91GHhcN3ViTitrMVh2Vi82V3l6a2tpaGFWWnBLYlhZPQ',
  'metadata': {'topic': 'debugging_and_tracing', 'type': 'query'},
  'messages': [{'role': 'User',
    'content': 'Hi, I am getting error running the code: The generated image exceeds the limit. How to solve it?\n```python\nPaddleHub2.4.0, PaddlePaddle2.5.2\nSystem environment: Windows\npython version: 3.9.18\nError report: In the prediction code example, an error is reported after running images = module.generate_image(text_prompts=text_prompts, style=\'oil painting\', output_dir=\'./ernie_vilg_out/\')\nTraceback (most recent call last):\nFile "", line 1, in\nFile "C:\\Users\\k\\anaconda3\\envs\\wm\\lib\\site-packages\\paddlehub\\compat\\paddle_utils.py", line 221, in runner\nreturn func(*args, **kwargs)\nFile "C:\\Users\\k.paddlehub\\modules\\ernie_vilg\\module.py", line 121, i

In [23]:
review_all_notebooks(parsed_notebooks)

Reviewing Notebooks:   0%|          | 0/13 [00:00<?, ?it/s]

[{'id': '1UeXB6ceNocGOSwSsOxH3ZQ2ZPFCBsQQn',
  'colab_link': 'https://colab.research.google.com/drive/1UeXB6ceNocGOSwSsOxH3ZQ2ZPFCBsQQn',
  'revision_id': '0BxmKQQ91GHhcV2ZiWlJKM0g4WHkwVXkzMUhuUVg1Q1hselpJPQ',
  'metadata': {'topic': 'database > transaction_management',
   'type': 'query',
   'target_turns': '2'},
  'messages': [{'role': 'User',
    'content': 'How do I start a transaction in SQLAlchemy?',
    'type': 'markdown'},
   {'role': 'Assistant',
    'content': 'There are actually 2 types of SQLAlchemy transactions\n\n\n1.   Core (low level SQL query)\n2.   ORM ( Objects relationship mapping)\n\n\nHere are the steps to start transaction in SQLAlchemy Core.\n\nSTEPS\n\n1.   Create an Engine\n\n     An `Engine` is a source of connectivity to a particular database.\n\n2.   Establish Connection\n\n     Use the `connect()` method of the `Engine` to establish a connection to the database.\n\n3.   Begin transaction\n\n     Use the `begin()` method on the connection object to start a 

In [18]:
print(notebooks_df['full_history_formatted_review'].loc[8])

# Critical Severity Issues
# Medium Severity Issues
# Scoring Explanation
The conversation provided does not contain any code generated by the AI Assistant for review. The user has presented an issue with their code, and the Assistant has only asked for more information to better understand the problem. Since there is no code from the Assistant to evaluate, the conversation cannot be scored based on the code quality criteria.

# Score: 5




In [21]:
import pandas as pd
from utils import DATA_DIR

def create_notebooks_dataframe(parsed_notebooks, selected_rows_df):
    """
    Create a dataframe from parsed notebooks and selected rows with additional processing.

    Args:
    parsed_notebooks (list): List of parsed notebooks.
    selected_rows_df (DataFrame): DataFrame of selected rows.

    Returns:
    DataFrame: The resulting dataframe after merging and processing.
    """
    notebooks_df = pd.DataFrame(parsed_notebooks)
    notebooks_df['full_history_code_score'] = notebooks_df['raw_review'].apply(
        lambda x: x.get('score') if x is not None else None
    )
    notebooks_df.rename(columns={'formatted_review': 'full_history_formatted_review'}, inplace=True)
    notebooks_df = notebooks_df.merge(
        selected_rows_df[['file_id', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']],
        left_on='id', right_on='file_id', how='left'
    )
    notebooks_df = notebooks_df[[
        'colab_link', 'full_history_formatted_review', 'full_history_code_score',
        'Code Quality', 'Language Quality', 'Feedback for the Trainer'
    ]]
    notebooks_df.rename(
        columns={
            'score': 'selected_rows_score',
            'Code Quality': 'Human Code Quality',
            'Language Quality': 'Human Language Quality',
            'Feedback for the Trainer': 'Human Reviewer Feedback'
        },
        inplace=True
    )
    notebooks_df['colab_link'] = notebooks_df['colab_link'].str.replace('?', '#')
    return notebooks_df

# Example usage:
# new_df = create_notebooks_dataframe(parsed_notebooks, selected_rows_df)
# new_df.to_csv(DATA_DIR + 'full_history_df_code_only.csv', index=False)
# print(new_df)


In [22]:
import pandas as pd
from utils import DATA_DIR

def create_notebooks_dataframe(parsed_notebooks, selected_rows_df):
    """
    Create a dataframe from parsed notebooks and selected rows with additional processing.

    Args:
    parsed_notebooks (list): List of parsed notebooks.
    selected_rows_df (DataFrame): DataFrame of selected rows.

    Returns:
    DataFrame: The resulting dataframe after merging and processing.
    """
    notebooks_df = pd.DataFrame(parsed_notebooks)
    notebooks_df['full_history_code_score'] = notebooks_df['raw_review'].apply(
        lambda x: x.get('score') if x is not None else None
    )
    notebooks_df.rename(columns={'formatted_review': 'full_history_formatted_review'}, inplace=True)
    notebooks_df = notebooks_df.merge(
        selected_rows_df[['file_id', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']],
        left_on='id', right_on='file_id', how='left'
    )
    notebooks_df = notebooks_df[[
        'colab_link', 'full_history_formatted_review', 'full_history_code_score',
        'Code Quality', 'Language Quality', 'Feedback for the Trainer'
    ]]
    notebooks_df.rename(
        columns={
            'score': 'selected_rows_score',
            'Code Quality': 'Human Code Quality',
            'Language Quality': 'Human Language Quality',
            'Feedback for the Trainer': 'Human Reviewer Feedback'
        },
        inplace=True
    )
    notebooks_df['colab_link'] = notebooks_df['colab_link'].str.replace('?', '#')
    return notebooks_df

# Example usage:
# new_df = create_notebooks_dataframe(parsed_notebooks, selected_rows_df)
# new_df.to_csv(DATA_DIR + 'full_history_df_code_only.csv', index=False)
# print(new_df)


In [45]:
import pandas as pd

notebooks_df = pd.DataFrame(parsed_notebooks)
notebooks_df['full_history_code_score'] = notebooks_df['raw_review'].apply(lambda x: x.get('score') if x is not None else None)
notebooks_df.rename(columns={'formatted_review': 'full_history_formatted_review'}, inplace=True)
notebooks_df = notebooks_df.merge(selected_rows_df[['file_id', 'Code Quality', 'Language Quality', 'Feedback for the Trainer', 'task_link_rev']], left_on='id', right_on='file_id', how='left')

notebooks_df = notebooks_df[['colab_link', 'task_link_rev', 'full_history_formatted_review', 'full_history_code_score', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']]

notebooks_df.rename(columns={'score': 'selected_rows_score', 'Code Quality':'Human Code Quality', 'Language Quality':'Human Language Quality', 'Feedback for the Trainer': 'Human Reviewer Feedback'}, inplace=True)
notebooks_df['colab_link'] = notebooks_df['colab_link'].str.replace('?', '#')
from utils import DATA_DIR
notebooks_df.to_csv(DATA_DIR + 'full_hsitory_df_code_only.csv', index=False)
notebooks_df

,colab_link,task_link_rev,full_history_formatted_review,full_history_code_score,Human Code Quality,Human Language Quality,Human Reviewer Feedback
0,https://colab.research.google.com/drive/1UeXB6...,https://colab.research.google.com/drive/1UeXB6...,# Critical Severity Issues\n# Medium Severity ...,4.0,3,3,"- missing code example in first turn, a sample..."
1,https://colab.research.google.com/drive/1qJPPq...,https://colab.research.google.com/drive/1qJPPq...,# Critical Severity Issues\n# Medium Severity ...,NaN,1,1,--It's empty.
2,https://colab.research.google.com/drive/1cP6qz...,https://colab.research.google.com/drive/1cP6qz...,# Critical Severity Issues\n# Medium Severity ...,3.0,2,1,-- No Explanation\n-- No Headers\n-- No Comments
3,https://colab.research.google.com/drive/1CPEz-...,https://colab.research.google.com/drive/1CPEz-...,# Critical Severity Issues\n# Medium Severity ...,5.0,NaN,NaN,NaN
4,https://colab.research.google.com/drive/189Fne...,https://colab.research.google.com/drive/189Fne...,# Critical Severity Issues\n# Medium Severity ...,3.0,NaN,NaN,NaN
5,https://colab.research.google.com/drive/1Unbsf...,https://colab.research.google.com/drive/1Unbsf...,# Critical Severity Issues\n# Medium Severity ...,4.0,NaN,NaN,NaN
6,https://colab.research.google.com/drive/1KtSOM...,https://colab.research.google.com/drive/1KtSOM...,# Critical Severity Issues\n# Medium Severity ...,4.0,NaN,NaN,NaN
7,https://colab.research.google.com/drive/1_lUJ5...,https://colab.research.google.com/drive/1_lUJ5...,# Critical Severity Issues\n# Medium Severity ...,NaN,1,1,-- assistant header is wrong it should be **As...
8,https://colab.research.google.com/drive/1JUY93...,https://colab.research.google.com/drive/1JUY93...,# Critical Severity Issues\n# Medium Severity ...,NaN,2,2,"-- really low effort, I understand that someti..."
9,https://colab.research.google.com/drive/1kL_7R...,https://colab.research.google.com/drive/1kL_7R...,# Critical Severity Issues\n# Medium Severity ...,NaN,1,1,-- The user question is clearly a copy and pas...


In [44]:
print(notebooks_df.loc[5]['task_link_rev'])

https://colab.research.google.com/drive/1Unbsf2xICxBcqMFzYBbyS9NW4kVtr7Os


In [42]:
print(notebooks_df.loc[8]['task_link_rev'])

https://colab.research.google.com/drive/1JUY9363HPsn5ChuweH_i31EzSsJPzSwX#revisionId=0BxmKQQ91GHhcN3ViTitrMVh2Vi82V3l6a2tpaGFWWnBLYlhZPQ


In [43]:
print(notebooks_df.loc[0]['task_link_rev'])

https://colab.research.google.com/drive/1UeXB6ceNocGOSwSsOxH3ZQ2ZPFCBsQQn#revisionId=0BxmKQQ91GHhcV2ZiWlJKM0g4WHkwVXkzMUhuUVg1Q1hselpJPQ


In [34]:
print(notebooks_df.loc[8]['full_history_formatted_review'])

# Critical Severity Issues
# Medium Severity Issues
# Scoring Explanation
The conversation provided does not include any code generated by the AI Assistant. The user has presented an issue with their code, and the Assistant has only asked for more information to better understand the problem. Since there is no code to evaluate, the criteria for code correctness, efficiency, best practices, readability, style consistency, and purpose cannot be applied. Therefore, the conversation cannot be scored based on the code quality rubric.

# Score: None




In [37]:
parsed_notebooks[6]

{'id': '1KtSOMVTE5kj8VuId-zm-y_TZVVMQwYwZ',
 'colab_link': 'https://colab.research.google.com/drive/1KtSOMVTE5kj8VuId-zm-y_TZVVMQwYwZ',
 'revision_id': None,
 'metadata': {'topic': 'debugging_and_tracing', 'target_turns': '1-4'},
 'messages': [{'role': 'User',
   'content': "Hi there, I'm prepping for a Smart City Planner interview. Part of my role will involve building a predictive maintenance system for urban infrastructure. I’m planning to use scikit-learn to develop the model but I'm getting stuck on fine-tuning. Any tips?",
   'type': 'markdown'},
  {'role': 'Assistant',
   'content': "Sure thing! Fine-tuning a machine learning model can be quite nuanced. Can you tell me what kind of performance metrics you're looking at and what specifically you're seeing with your model's output? Any details on the dataset you’re using will also be helpful.",
   'type': 'markdown'},
  {'role': 'User',
   'content': "I'm using the dataset that includes historical utility data like usage patterns,

In [23]:
import pandas as pd

notebooks_df = pd.DataFrame(parsed_notebooks)
notebooks_df['full_history_code_score'] = notebooks_df['raw_review'].apply(lambda x: x.get('score') if x is not None else None)
notebooks_df.rename(columns={'formatted_review': 'full_history_formatted_review'}, inplace=True)
notebooks_df = notebooks_df.merge(selected_rows_df[['file_id', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']], left_on='id', right_on='file_id', how='left')

notebooks_df = notebooks_df[['colab_link', 'full_history_formatted_review', 'full_history_code_score', 'Code Quality', 'Language Quality', 'Feedback for the Trainer']]

notebooks_df.rename(columns={'score': 'selected_rows_score', 'Code Quality':'Human Code Quality', 'Language Quality':'Human Language Quality', 'Feedback for the Trainer': 'Human Reviewer Feedback'}, inplace=True)
notebooks_df['colab_link'] = notebooks_df['colab_link'].str.replace('?', '#')
from utils import DATA_DIR
notebooks_df.to_csv(DATA_DIR + 'full_hsitory_df_code_only.csv', index=False)
notebooks_df

,colab_link,full_history_formatted_review,full_history_code_score,Human Code Quality,Human Language Quality,Human Reviewer Feedback
0,https://colab.research.google.com/drive/1UeXB6...,# Critical Severity Issues\n# Medium Severity ...,3,3,3,"- missing code example in first turn, a sample..."
1,https://colab.research.google.com/drive/1qJPPq...,# Critical Severity Issues\n# Medium Severity ...,5,1,1,--It's empty.
2,https://colab.research.google.com/drive/1cP6qz...,# Critical Severity Issues\n# Medium Severity ...,3,2,1,-- No Explanation\n-- No Headers\n-- No Comments
3,https://colab.research.google.com/drive/1CPEz-...,# Critical Severity Issues\n# Medium Severity ...,4,NaN,NaN,NaN
4,https://colab.research.google.com/drive/189Fne...,# Critical Severity Issues\n# Medium Severity ...,3,NaN,NaN,NaN
5,https://colab.research.google.com/drive/1Unbsf...,# Critical Severity Issues\n# Medium Severity ...,4,NaN,NaN,NaN
6,https://colab.research.google.com/drive/1KtSOM...,# Critical Severity Issues\n# Medium Severity ...,3,NaN,NaN,NaN
7,https://colab.research.google.com/drive/1_lUJ5...,# Critical Severity Issues\n# Medium Severity ...,5,1,1,-- assistant header is wrong it should be **As...
8,https://colab.research.google.com/drive/1JUY93...,# Critical Severity Issues\n# Medium Severity ...,5,2,2,"-- really low effort, I understand that someti..."
9,https://colab.research.google.com/drive/1kL_7R...,# Critical Severity Issues\n# Medium Severity ...,5,1,1,-- The user question is clearly a copy and pas...
